In [1]:
## include your gdrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


This is the environment setup for using colab 

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
!pip install bert-serving-client
!pip install -U bert-serving-server[http]

     |████████████████████████████████| 61 kB 229 kB/s 
     |████████████████████████████████| 357 kB 20.9 MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=3cb79c712fda80dcbd24a2a67ab71a12c7bf1aad28175a3a8741835e4c0ca1dd
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil


In [4]:
!nohup bert-serving-start -model_dir=/content/gdrive/MyDrive/uncased_L-12_H-768_A-12 > out.file 2>&1 &

In [5]:
!pip install faiss-cpu

     |████████████████████████████████| 8.4 MB 7.6 MB/s 


# Text Augmentation using Bert Embedding

In [6]:
def create_answer_corpus(data_address):
  f = open(data_address, "r")
  lines = [x.strip() for x in f.readlines()]

  answers_corpus = []
  for (i, line) in enumerate(lines):
    group = line.split("\t")
    ans_candidate = group[2]
    if ans_candidate not in answers_corpus:
      answers_corpus.append(ans_candidate)
  return answers_corpus

def create_ans_embedding(answers_corpus, bc):
  embedded_ans = bc.encode(ans_corpus)

  ans_emb = {}

  for i,sentence in enumerate(answers_corpus):
    ans_emb.update({ sentence: embedded_ans[i] })
  
  return ans_emb

def create_mixup_data(data_address,top_k, ans_emb, d, new_data_address):
  # Data_address: the address of the data you want to augment
  # top_k: finding the top k most similar answers candidate to the real answer and give them the same label
  # ans_emb: is a dictionray that has all the candidate answers as keys and their corresponding embeddings as values
  # d: is the number of dimension for the word embedding
  # new_data_address: is the stroing address of the augmented result

  from scipy.spatial.distance import pdist

  def calculate_cosine_sim_between_two_idx(corpus, id1, id2, mixup = False):
    return pdist([corpus[id1], corpus[id2]], metric="cosine").mean()

  d=d
  ans_emb_val = list(ans_emb.values())
  index=faiss.IndexFlatIP(d)
  index.add(np.array(ans_emb_val))
  D, I=index.search(np.array(ans_emb_val), top_k+1)
  ## 
  threshold = pdist(ans_emb_val, metric="cosine").mean()
  print(threshold)

  # store all the candidate into a dictionary
  from operator import itemgetter 
  anss = list(ans_emb.keys())
  ans_words_sim = {}
  ans_idxs_sim = {}

  for i,idxs in enumerate(I):
    
    cur_word_idx = np.int64(i)
    temp = []
    for idx in enumerate(idxs):
      if idx != i:
        temp.append(idx[1])

    most_similar = np.array(temp, dtype=int)



    ans_words_sim.update({ anss[cur_word_idx]: itemgetter(*most_similar)(anss) })
    ans_idxs_sim.update({ anss[cur_word_idx]: [i] + temp }) 

  f = open(data_address, "r")
  lines = [x.strip() for x in f.readlines()]

  current_q = None

  new_f = open(new_data_address, "w")
  # create psuedo data for the same type
  for (i, line) in enumerate(lines):
    group = line.split("\t")
    text_a = group[0] + " " + group[1]
    text_b = group[2]
    label = group[3]
    cat = group[4]
    # if current_q == None:
    #   currenet_q = text_a
    pseudo_candidates = ans_words_sim[text_b]
    cur_ans_idx = ans_idxs_sim[text_b][0]
    pseudo_candidates_idx = ans_idxs_sim[text_b][1:]

    cur_line = str(group[0]+"\t"+group[1]+"\t"+text_b+"\t"+label+"\t"+cat)
    new_f.write(cur_line)
    new_f.write("\n")

    for i, candidate in enumerate(pseudo_candidates):
      if calculate_cosine_sim_between_two_idx(ans_emb_val,cur_ans_idx ,pseudo_candidates_idx[i]) <= threshold:
        new_candidate_line = str(group[0]+"\t"+group[1]+"\t"+candidate+"\t"+label+"\t"+cat)
        new_f.write(new_candidate_line)
        new_f.write("\n")

  new_f.close() 
  # create psuedo data using mixup
  


In [ ]:
from bert_serving.client import BertClient
bc = BertClient()

In [10]:
import faiss
import numpy as np
for i in range(5):
  ans_corpus = create_answer_corpus(f"/content/gdrive/MyDrive/light_MCTACO/Temporal_commonsense_reasoning/fold_{i+1}.tsv")
  ans_emb = create_ans_embedding(ans_corpus,bc)
  create_mixup_data(f"/content/gdrive/MyDrive/light_MCTACO/Temporal_commonsense_reasoning/fold_{i+1}.tsv",5, ans_emb, 768, f"/content/gdrive/MyDrive/light_MCTACO/Temporal_commonsense_reasoning/augment_fold_{i+1}.tsv")


0.40608993133704124
0.41209025606350286
0.40992398821254705
0.4085991274346685
0.4135460102639389
